<a href="https://colab.research.google.com/github/CgriefTesla/Dairy-of-algorithm/blob/main/Text_mining_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing some necessary lib

In [ ]:
## importing lib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import collections

Downloading some model

In [ ]:
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("brown")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Checking the data in Brown.

The Corpus consists of 500 samples, distributed across 15 genres in rough proportion to the amount published in 1961 in each of those genres. All works sampled were published in 1961; as far as could be determined they were first published then, and were written by native speakers of American English.

Each sample began at a random sentence-boundary in the article or other unit chosen, and continued up to the first sentence boundary after 2,000 words. In a very few cases miscounts led to samples being just under 2,000 words.

The original data entry was done on upper-case only keypunch machines; capitals were indicated by a preceding asterisk, and various special items such as formulae also had special codes.

The corpus originally (1961) contained 1,014,312 words sampled from 15 text categories:

  

*   A. PRESS: Reportage (44 texts)

    1 Political

    2 Sports

    3 Society

    4 Spot News

    5 Financial

    6 Cultural

*   B. PRESS: Editorial (27 texts)

    1 Institutional Daily

    2 Personal

    3 Letters to the Editor

*   C. PRESS: Reviews (17 texts)

    1 theatre

    2 books

    3 music

    4 dance
*   D. RELIGION (17 texts)

    1 Books

    2 Periodicals

    3 Tracts
*   E. SKILL AND HOBBIES (36 texts)

    1 Books

    2 Periodicals
*   F. POPULAR LORE (48 texts)

    1 Books

    2 Periodicals
*   G. BELLES-LETTRES - Biography, Memoirs, etc. (75 texts)

    1 Books

    2 Periodicals
*   H. MISCELLANEOUS: US Government & House Organs (30 texts)

    1 Government Documents

    2 Foundation Reports

    3 Industry Reports

    4 College Catalog

    5 Industry House organ
*   J. LEARNED (80 texts)

    1 Natural Sciences

    2 Medicine

    3 Mathematics

    4 Social and Behavioral Sciences

    5 Political Science, Law, Education

    6 Humanities

    7 Technology and Engineering
*   K. FICTION: General (29 texts)

    1 Novels

    2 Short Stories
*   L. FICTION: Mystery and Detective Fiction (24 texts)

    1 Novels

    2 Short Stories
*   M. FICTION: Science (6 texts)

    1 Novels

    2 Short Stories
*   N. FICTION: Adventure and Western (29 texts)

    1 Novels

    2 Short Stories
*   P. FICTION: Romance and Love Story (29 texts)

    1 Novels

    2 Short Stories
*   R. HUMOR (9 texts)

    1 Novels

    2 Essays, etc.




In [ ]:
from nltk.corpus import brown as corpus

for n,item in enumerate(corpus.words(corpus.fileids()[0])[:300]):
    print(item, end=" ")
    if (n%25) ==24:
      print(" ")

cn23
The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place .  
The jury further said in term-end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise  
and thanks of the City of Atlanta '' for the manner in which the election was conducted . The September-October term jury had been charged  
by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won by Mayor-nominate Ivan  
Allen Jr. . `` Only a relative handful of such reports was received '' , the jury said , `` considering the widespread interest in  
the election , the number of voters and the size of this city '' . The jury said it did find that many of Georgia's  
registration and election laws `` are outmoded or inadequate and often ambiguous '' . It recommended that Fulton legislators 

## Checking the texts in documents

In [ ]:
## The number of documents
len(corpus.fileids())

500

In [ ]:
docs=[corpus.words(fileid) for fileid in corpus.fileids()]
print(corpus.words('ca01'))
print('----------------')
print(docs[:5])
print("num of docs:", len(docs))

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]
----------------
[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...], ['Austin', ',', 'Texas', '--', 'Committee', 'approval', ...], ['Several', 'defendants', 'in', 'the', 'Summerdale', ...], ['Oslo', 'The', 'most', 'positive', 'element', 'to', ...], ['East', 'Providence', 'should', 'organize', 'its', ...]]
num of docs: 500


## get the stop words

In [ ]:
## English stopwords

# English stopwords defined by the NLTK package.
en_stop = nltk.corpus.stopwords.words('english')

## Data preprocess


*   coverting to lowercase
*   remove some stopwords and  ','
*   lemmatize



In [ ]:
## data preprocess

from nltk.corpus import wordnet as wn # import for lemmatize

def preprocess_word(word, stopwordset):
    
    #1.convert words to lowercase (e.g., Python =>python)
    word=word.lower()
    
    #2.remove "," and "."
    if word in [",","."]:
        return None
    
    #3.remove stopwords  (e.g., the => (None)) 
    if word in stopwordset:
        return None
    
    #4.lemmatize  (e.g., cooked=>cook)
    lemma = wn.morphy(word)
    if lemma is None:
        return word

    # lemmatized words could be in the stopwords set
    elif lemma in stopwordset: 
        return None
    else:
        return lemma
    

def preprocess_document(document):
    document=[preprocess_word(w, en_stop) for w in document]
    document=[w for w in document if w is not None]
    return document

def preprocess_documents(documents):
    return [preprocess_document(document) for document in documents]

## Checking the function using the first document

In [ ]:
## check the function of data preprocess

# before
print(docs[0][:25]) 

# after
print(preprocess_documents(docs)[0][:25])

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']
['fulton', 'county', 'grand', 'jury', 'say', 'friday', 'investigation', "atlanta's", 'recent', 'primary', 'election', 'produce', '``', 'evidence', "''", 'irregularity', 'take', 'place', 'jury', 'say', 'term-end', 'presentment', 'city', 'executive', 'committee']


## Calculating the documents embedding(vector).

Here I use tf-idf vectorizer to process the document

Ignoring the one letter words

Setting the max_feature to the Square root of the number of words, 1000

In [ ]:
## clustering
# define the vectorizer
pre_docs=preprocess_documents(docs)
pre_docs=[" ".join(doc) for doc in pre_docs]
print(pre_docs[0])

vectorizer = TfidfVectorizer(max_features=1000, token_pattern=u'(?u)\\b\\w\\w+\\b' )


# fit
tf_idf = vectorizer.fit_transform(pre_docs)

fulton county grand jury say friday investigation atlanta's recent primary election produce `` evidence '' irregularity take place jury say term-end presentment city executive committee over-all charge election `` deserve praise thanks city atlanta '' manner election conduct september-october term jury charge fulton superior court judge durwood pye investigate report possible `` irregularity '' hard-fought primary mayor-nominate ivan allen jr. `` relative handful report receive '' jury say `` consider widespread interest election number voter size city '' jury say find many georgia's registration election laws `` outmode inadequate often ambiguous '' recommend fulton legislator act `` laws study revise end modernize improve '' grand jury comment number topic among atlanta fulton county purchasing department say `` well operate follow generally accept practice inure best interest government '' merger propose however jury say belief `` two office combine achieve greater efficiency reduce

## K-means with Euclidean distance

## convert the categories into int list

In [ ]:
print('This is categories:',corpus.categories(corpus.fileids()))

labels = corpus.fileids()
print('This is raw ground truth labels:',labels)
from string import digits
remove_digits = str.maketrans('', '', digits)
labels_l = []
for label in labels:
  label = label.translate(remove_digits)
  labels_l.append(label)
print('This is processed ground truth labels:', labels_l)

dic = {'ca':0,'cb':1,'cc':2, 'cd':3,'ce':4, 'cf':5,'cg':6,'ch':7, 'cj':8,'ck':9,'cl':10, 'cm':11,'cn':12,'cp':13,'cr': 14}
true_labels = []

for i in labels_l:
  true_labels.append(dic[i])
print('This is converted ground truth:',true_labels)

This is categories: ['adventure', 'belles_lettres', 'editorial', 'fiction', 'government', 'hobbies', 'humor', 'learned', 'lore', 'mystery', 'news', 'religion', 'reviews', 'romance', 'science_fiction']
This is raw ground truth labels: ['ca01', 'ca02', 'ca03', 'ca04', 'ca05', 'ca06', 'ca07', 'ca08', 'ca09', 'ca10', 'ca11', 'ca12', 'ca13', 'ca14', 'ca15', 'ca16', 'ca17', 'ca18', 'ca19', 'ca20', 'ca21', 'ca22', 'ca23', 'ca24', 'ca25', 'ca26', 'ca27', 'ca28', 'ca29', 'ca30', 'ca31', 'ca32', 'ca33', 'ca34', 'ca35', 'ca36', 'ca37', 'ca38', 'ca39', 'ca40', 'ca41', 'ca42', 'ca43', 'ca44', 'cb01', 'cb02', 'cb03', 'cb04', 'cb05', 'cb06', 'cb07', 'cb08', 'cb09', 'cb10', 'cb11', 'cb12', 'cb13', 'cb14', 'cb15', 'cb16', 'cb17', 'cb18', 'cb19', 'cb20', 'cb21', 'cb22', 'cb23', 'cb24', 'cb25', 'cb26', 'cb27', 'cc01', 'cc02', 'cc03', 'cc04', 'cc05', 'cc06', 'cc07', 'cc08', 'cc09', 'cc10', 'cc11', 'cc12', 'cc13', 'cc14', 'cc15', 'cc16', 'cc17', 'cd01', 'cd02', 'cd03', 'cd04', 'cd05', 'cd06', 'cd07', 'cd08

## Building the function to calculate the purity score

In [ ]:
import numpy as np
from sklearn import metrics

def purity_score(y_true, y_pred):
    # compute contingency matrix (also called confusion matrix)
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    print('This is confusion matrix')
    print('-------------------')
    print(contingency_matrix)
    print('-------------------')
    # return purity
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix) 

In [ ]:
# K-means setting

num_clusters = 15
km = KMeans(n_clusters=num_clusters, random_state = 0)

# fit
clusters = km.fit_predict(tf_idf)

## getting the predicted_labels

In [ ]:
pre_labels = []
for doc, cls in zip(preprocess_documents(docs), clusters):
    print(cls)
    pre_labels.append(cls)

In [ ]:
print('This is predicted labels:',pre_labels)
print('This is ground truth labels:', true_labels)

print('This is purity score:',purity_score(true_labels, pre_labels))


This is predicted labels: [4, 4, 11, 4, 2, 4, 4, 4, 4, 4, 13, 13, 13, 13, 13, 9, 9, 9, 2, 4, 9, 9, 9, 2, 4, 11, 4, 11, 9, 9, 6, 13, 9, 4, 0, 4, 4, 13, 13, 13, 4, 4, 11, 11, 4, 4, 4, 4, 4, 4, 4, 10, 2, 1, 4, 4, 1, 4, 11, 4, 4, 4, 4, 4, 4, 6, 4, 13, 4, 0, 1, 2, 13, 2, 1, 1, 1, 2, 13, 2, 7, 2, 2, 2, 2, 13, 4, 1, 7, 6, 6, 6, 6, 6, 1, 7, 6, 5, 7, 5, 13, 6, 7, 6, 6, 2, 13, 4, 1, 9, 11, 8, 8, 13, 8, 12, 13, 13, 8, 8, 8, 8, 13, 13, 11, 11, 1, 7, 12, 11, 5, 8, 11, 0, 11, 11, 11, 11, 11, 11, 11, 1, 12, 7, 2, 12, 10, 1, 1, 10, 9, 1, 1, 11, 5, 6, 12, 13, 1, 5, 4, 13, 12, 4, 2, 0, 13, 13, 2, 4, 0, 9, 1, 0, 11, 12, 12, 6, 13, 1, 4, 13, 2, 13, 6, 4, 4, 2, 6, 2, 5, 4, 12, 13, 1, 4, 2, 7, 4, 1, 7, 7, 12, 7, 7, 2, 7, 7, 11, 5, 5, 1, 4, 7, 1, 7, 2, 5, 1, 1, 9, 1, 1, 5, 4, 12, 4, 1, 2, 9, 5, 7, 7, 4, 5, 10, 12, 1, 12, 10, 4, 10, 10, 6, 7, 0, 4, 1, 10, 5, 4, 7, 2, 5, 10, 10, 0, 7, 1, 7, 5, 7, 1, 10, 11, 11, 4, 11, 11, 11, 11, 4, 11, 11, 14, 4, 4, 11, 8, 11, 4, 4, 11, 11, 11, 11, 11, 11, 4, 2, 11, 0, 11, 0,

# Content.

 I used brown dataset to do the K-means text mining. The details of the dataset is above.

# similarity criteria.

I converted the documents into tf-idf vectors, and calculated their Euclidean distance.

# Regarding to the different K.

Exactly I didn't use different K to compare the result, because this dataset has their own ground truth. So I don't think it is suitable to change the number of classes, and also if the classes was changed I don't know how to evaluate the result.

# contents of clusters.

I think it is better to see the confusion matrix above. In that matrix you can get the information about my result.

# Evaluating

The purity score of this model is 0.37 and inverse purity score is 0.39 (you could get that by changing the parameter of purity_score function. changing 'axis=0' to 'axis=1')

# Consideration

I don't think it is a good result, but I don't how to improve it. I think the point is that the dataset **Brown** is classified by their type of literature but not the content. So the struct of the document is more important than the content. 